<a href="https://colab.research.google.com/github/hesller/python-ai/blob/main/IQ_Option_v003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goals

*  fazer um loop sobre a lista
 * pegar index da trade  a ser feita
 * salvar esse index
 * calcular tempo que falta para  
  * tempo na tabela menos tempo atual
  * botar o computador pra dormir essa quantidade de tempo

Keep colab running:

function ClickConnect(){
console.log("Working"); 
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ClickConnect,60000)

In [ ]:
!sudo pip3 install -U git+git://github.com/Lu-Yi-Hsun/iqoptionapi.git

In [ ]:
import pandas as pd
import numpy as np
import time
import os
from iqoptionapi.stable_api import IQ_Option
import logging
from datetime import datetime
import asyncio

logging.disable(level=logging.CRITICAL)

In [ ]:
os.environ['TZ'] = 'Brazil/East'
time.tzset()

### Make the login and setup server constants


In [ ]:
start = time.perf_counter()
iqapi = IQ_Option("eng.hesller@gmail.com","IQZinho1508!")
iqapi.connect()
finish = time.perf_counter()
print(finish - start)

2.857602758000013


In [ ]:
iqapi.change_balance('REAL')
iqapi.get_balance()

73.32

### Get the data


In [ ]:
# get the dictionary of trades
trades_dict = pd.read_csv('/content/19-03-2020_15min.csv', sep=';')
trades_dict['asset'] = trades_dict['asset'].str.replace(r'/','')

trades_dict

### Helper functions


In [ ]:
def get_secs_left(dataframe, trade_index):
  curr_time = time.localtime()
  curr_secs = curr_time.tm_hour * 60 * 60 + curr_time.tm_min * 60 + curr_time.tm_sec
  trade_secs = dataframe['hour'][trade_index] * 60 * 60 + dataframe['min'][trade_index] * 60

  return trade_secs - curr_secs

def organize_duration(dataset):
    ds = dataset.copy()
    for i in range(len(dataset['time'])):
        x = dataset['time'][i]
        ds.loc[i, 'hour'] = int(x[0:2])
        ds.loc[i, 'min'] = int(x[3:5])
        ds.loc[i, 'sec'] = int(x[6:])
    duration = {
        'M5': 1,
        'M5': 5,
        'M15': 15,
        'M30': 30,
        'H1': 60,
    }
    for i in range(len(ds)):
        str_dur = dataset['duration'][i]
        ds.loc[i, 'duration'] = duration[str_dur]

    return ds


class TradeAgent:
  def __init__(self, trades_dict = {}, start_balance = 0, 
               initial_trade_size = 1,trade_type = 'binary',
               size_streaming=1, mult_factor=1.75, max_mart=1,
               stop_loss=12, max_profit=50, use_soros=True,
               max_soros=1, digital_win_guess=0.85, mart_soros_draw =True, duration=5, name='', api='iqapi'):
    
    self.trades_dict = trades_dict
    self.starting_balance = start_balance
    self.trade_index = 0
    self.trade_type = trade_type
    self.curr_trade_id = 0
    self.trade_duration = duration
    self.size_streaming = 1
    self.initial_trade_size = initial_trade_size
    self.trade_size = self.initial_trade_size
    self.mult_factor = mult_factor
    self.max_mart = max_mart
    self.curr_mart = 0
    self.use_mart = True
    self.profit_loss = 0
    self.server_win_pct = 0.75
    self.stop_loss = stop_loss
    self.max_profit = max_profit
    self.use_soros = use_soros
    self.curr_soros = 0
    self.max_soros = max_soros
    self.digital_win_pct = digital_win_guess
    self.mart_soros_on_draw = mart_soros_draw
    self.next_soros = True
    self.last_trade_won = False
    self.name = name
    self.iqapi = api

  def set_name(self, name):
    self.name = name

  def set_api(self, api):
    self.iqapi = api

  def set_asset_and_order(self):
    self.curr_asset = str(self.trades_dict['asset'][self.trade_index])
    self.curr_order = str(self.trades_dict['order'][self.trade_index]).lower()
    self.trade_duration = self.trades_dict['duration'][self.trade_index]
  
  def set_mult_factor(self, factor):
    self.mult_factor = factor

  def set_trade_size(self):
    self.trade_size *= self.mult_factor
  
  def set_trade_index(self):
    self.trade_index += 1
  
  def set_last_trade_won(self, condition):
    self.last_trade_won = condition
    return True

  def set_soros_trade_size(self):
    print('nothing to set on soros size')
  
  def set_profit_loss(self, loss):
    if loss:
      self.profit_loss -= self.trade_size
    else:
      self.profit_loss += self.trade_size * self.server_win_pct
    print('current profit/loss: ', self.profit_loss)

  def reset_trade_size(self):
    self.trade_size = self.initial_trade_size

  def reset_mart(self):
    self.curr_mart = 0
  
  def reset_soros(self):
    self.curr_soros = 0

  def open_trade(self):
      # open the trade
      print('- /n')
      print('======================')
      print('previous balance: ', balance)
      print('executing trade...')

      if self.trade_type == 'digital':
        print('digital option...')
        print(f'option: {self.curr_asset}-{self.trade_size}-{self.curr_order}-{self.trade_duration}')
        self.curr_trade_id = self.iqapi.buy_digital_spot(self.curr_asset, 
                                                    self.trade_size, 
                                                    self.curr_order, 
                                                    self.trade_duration)
        self.curr_trade_id = self.curr_trade_id[1]
        # TODO: adjust correcly the server_win_pct
        print('digital option requested')
        time.sleep(5)
        self.server_win_pct= self.digital_win_pct
      else:
        _, self.curr_trade_id = self.iqapi.buy(self.trade_size, self.curr_asset, 
                                          self.curr_order, 
                                          self.trade_duration)
        # TODO: adjust correcly the server_win_pct
        time.sleep(5)
        order_dict = self.iqapi.get_async_order(self.curr_trade_id)
        self.server_win_pct = (order_dict['msg']['profit_percent'] -100 ) / 100
      # salvando em self, não vou precisar retornar nada
      #return id, trade_index, asset,
      print('this is the trade id: ', self.curr_trade_id)
      if self.curr_trade_id <= 0:
        print('instrumento não disponível, trade ID ', self.curr_trade_id )
        return False
      else:
        print('ordem aberta. ID: ', self.curr_trade_id, ' - pct win: ', self.server_win_pct)
        return True
        
  def check_win_loss(self, cc, k):
    loss = False # for safety set it default to false
    #open_price = iqapi.get_async_order(self.curr_trade_id)['open_underlying_price']
    open_price = self.iqapi.get_async_order(self.curr_trade_id)
    if self.trade_type != 'digital': 
      open_price = open_price['msg']['value']
    else:
      open_price = open_price['position-changed']['msg']['open_quote']

    if self.curr_order == 'call':
      # check win
      if cc[k]['close'] > open_price:
        print(f'Agent_{self.name} você ganhou essa negociação.')
        loss = False
        return loss
      # check loss
      elif cc[k]['close'] < open_price:
        print(f'Agent_{self.name} você perdeu essa negociação')
        loss = True
        return loss
      # check draw
      else:
        print(f'Agent_{self.name} negociação deu empate.')
        if self.trade_type == 'digital':
          print(f'Agent_{self.name} o tipo de negociação é: ', self.trade_type)
          print(f'Agent_{self.name} assim sendo pelas regras da IQ Option você perdeu essa negociação')
          loss = True
          return loss
        else:
          if self.mart_soros_on_draw == True:
            loss = True
            self.reset_trade_size()
          else:
            loss = False
          return loss
    elif self.curr_order == 'put':
      #check win
      if cc[k]['close'] < open_price:
        print(f'Agent_{self.name} você ganhou essa negociação PUT.')
        loss = False
        return loss
      # check loss
      elif cc[k]['close'] > open_price:
        print(f'Agent_{self.name} você perdeu essa negociação PUT')
        loss = True
        return loss
      # check draw
      else:
        print(f'Agent_{self.name} negociação deu empate. PUT')
        if self.trade_type == 'digital':
          print(f'Agent_{self.name} o tipo de negociação é: ', self.trade_type)
          print(f'Agent_{self.name} assim sendo pelas regras da IQ Option você perdeu essa negociação. PUT')
          loss = True
          return loss
        else:
          if self.mart_soros_on_draw == True:
            print('o tipo de negociação é: ', self.trade_type)
            print(' e você configurou martingale/soros durante empate: ', self.mart_soros_on_draw)
            print(f'Agent_{self.name} assim sendo pelas regras da IQ Option você perdeu essa negociação PUT')
            loss = True
            self.reset_trade_size()
          else:
            loss = False
          return loss

  def start_streaming(self):
    print('================== /n')
    print(f'Agent_{self.name} Iniciando Stream de preços às: ', time.localtime())
    self.iqapi.start_candles_stream(self.curr_asset, self.size_streaming, 1)

  def candle_analysis(self):
    print(f'Agent_{self.name} analisando vela...  /n')
    time.sleep(2)
    checked = False
    while True:
      cc=self.iqapi.get_realtime_candles(self.curr_asset, self.size_streaming)
      for k in cc:
        curr_sec = int(datetime.fromtimestamp(cc[k]['from']).strftime("%S"))
        curr_min = int(datetime.fromtimestamp(cc[k]['from']).strftime("%M"))
        curr_time = curr_min * 60 + curr_sec + 1
        if curr_time % (self.trade_duration * 60) == 0:
          print(f'/n Agent_{self.name} verificando vitória ou derrota...')
          loss = self.check_win_loss(cc, k)
          checked = True
          break
      if checked:
        break
      time.sleep(1)
    return loss
  
  def open_martingale(self):
    if self.use_mart:
        self.curr_mart += 1
        if self.curr_mart <= self.max_mart:
          print('==================')
          print(f'Agent_{self.name} Martingale de número: ', self.curr_mart)
          print(f'Agent_{self.name} Fator mutiplicador: ', self.mult_factor)
          # TODO: PODERIA CHAMAR AQUI O SET_MULT_FACTOR PARA FATORES DIFERENTES
          # DE MARTINGALE
          self.set_trade_size()
          print(f'Agent_{self.name} Valor a ser negociado: ', self.trade_size)
          ## TODO: definir a função martingale
          self.open_trade()
          print(f'Agent_{self.name} Negociação Martingale iniciada com sucesso! ID: ', self.curr_trade_id)
  
  def open_soros(self):
    self.trade_size += self.server_win_pct * self.trade_size
    if self.use_soros:
        self.curr_soros += 1
        if self.curr_soros <= self.max_soros:
          print('==================')
          print('Soros de número: ', self.curr_soros)
          #print('Fator mutiplicador: ', self.mult_factor)
          # TODO: PODERIA CHAMAR AQUI O SET_MULT_FACTOR PARA FATORES DIFERENTES
          # DE MARTINGALE
          print('Valor a ser negociado: ', self.trade_size)
          ## TODO: definir a função martingale
          self.open_trade()
          print('Negociação Martingale iniciada com sucesso! ID: ', self.curr_trade_id)

  def stop_streaming(self):
    self.iqapi.stop_candles_stream(self.curr_asset, self.size_streaming)

  def is_final_break(self):
    if self.profit_loss < 0 and self.profit_loss <= self.stop_loss:
      print('infelizmente você alcançou o stop loss do dia.')
      print(' estarei desligando o algoritmo para evitar mais perdas')
      return True
    elif self.profit_loss > 0 and self.profit_loss >= self.max_profit:
      print('parabens a meta do dia foi alcançada...')
      print('desligando algoritmo.')
      return True
    else:
      return False

  def check_soros(self):
    print("nothing to do")

  def reset_defaults(self):
    self.reset_trade_size()
    self.reset_mart()
    self.reset_soros()

  ## TODO: ADRESS THE ERROR OF NON EXISTING PAIR AVAILABLE

### Main Loop

In [ ]:
trading = True
trade_index = 0
martingale = True
duration = 1
otc = False
digital = True
balance = iqapi.get_balance()
trades_dict = organize_duration(trades_dict)

In [ ]:
idx = 0
idx_equal_1 = []
idx_equal_2 = []
idx_equal_3 = []
new_ds = trades_dict

while idx < len(new_ds) - 4:
    sec = new_ds.loc[idx, 'hour'] * 3600 + new_ds.loc[idx, 'min'] * 60
    next_sec = new_ds.loc[idx+1, 'hour'] * 3600 + new_ds.loc[idx+1, 'min'] * 60
    next_2_sec = new_ds.loc[idx+2, 'hour'] * 3600 + new_ds.loc[idx+2, 'min'] * 60
    next_3_sec = new_ds.loc[idx+3, 'hour'] * 3600 + new_ds.loc[idx+3, 'min'] * 60

    if sec == next_sec:
        idx_equal_1.append(idx+1)
        if sec == next_2_sec:
            idx_equal_2.append(idx + 2)
            if sec == next_3_sec:
                idx_equal_3.append(idx + 3)
    idx += 1

second_ds = pd.DataFrame()
third_ds = pd.DataFrame()
forth_ds = pd.DataFrame()

second_ds = new_ds.loc[idx_equal_1]
third_ds = new_ds.loc[idx_equal_2]
forth_ds = new_ds.loc[idx_equal_3]

second_ds = second_ds.drop(idx_equal_2)
third_ds = third_ds.drop(idx_equal_3)

list1 = list(set(idx_equal_1) | set(idx_equal_2))
list2 = list(set(list1) | set(idx_equal_3))

first_ds = new_ds.drop(list2)

In [ ]:
ds_vector = [first_ds, second_ds, third_ds, forth_ds]
for ds in ds_vector:
  ds.set_index('time',inplace=True)
  ds.reset_index(inplace=True)
process_vector = [1, 2, 3, 4]

In [ ]:
timenow = time.localtime()
for dictio in ds_vector:
  for i in range(len(dictio)):
    if i < 2:
      dictio.loc[i, 'min'] = timenow.tm_min + 2
      dictio.loc[i, 'hour'] = timenow.tm_hour
      dictio.loc[i, 'asset'] = 'EURUSD'
    elif i <= 4:
      dictio.loc[i, 'min'] = timenow.tm_min + 4
      dictio.loc[i, 'hour'] = timenow.tm_hour
      dictio.loc[i, 'asset'] = 'USDJPY'
    elif i <= 8:
      dictio.loc[i, 'min'] = timenow.tm_min + 8
      dictio.loc[i, 'hour'] = timenow.tm_hour
      dictio.loc[i, 'asset'] = 'EURUSD'
    else:
      dictio.loc[i, 'min'] = timenow.tm_min + 8
      dictio.loc[i, 'hour'] = timenow.tm_hour
      dictio.loc[i, 'asset'] = 'USDJPY'

In [ ]:
### function loop

### Open process function

In [ ]:
def open_process(trades_dict, agent):
  process_number = agent.name
  
  iqapi = IQ_Option('eng.hesller@gmail.com', 'IQZinho1508!')
  iqapi.connect()
  iqapi.change_balance('REAL')
  print('the inicial balance: ', iqapi.get_balance())
  time.sleep(10)
  os.environ['TZ'] = 'Brazil/East'
  time.tzset()

  agent.set_api(iqapi)

  i = 0
  while i < len(trades_dict):
    i+=1
    # comput the seconds left for the next trade and sleep
    print(f'computing seconds left in process {process_number}...')
    if agent.trade_index >= len(trades_dict):
      msg = f' index fora de range... Finalizando {process_number}. Obrigado :)'
      break

    secs_left = get_secs_left(trades_dict, agent.trade_index)

    if secs_left < 0:
      agent.set_trade_index()
    if secs_left > 0:
      print(f'{process_number} sleeping for: {secs_left} seconds. ')
      time.sleep(np.abs(secs_left - 60))
      iqapi.connect()
    
      secs_left = get_secs_left(trades_dict, agent.trade_index)

      if secs_left < 0:
        agent.set_trade_index()
      if secs_left > 0:
        print(f'{process_number} sleeping for: {secs_left} seconds. ')
        time.sleep(np.abs(secs_left - 2))

      # criar uma função a partir daqui
      agent.set_asset_and_order()

      print('iniciando negociação normal')
      print('valor negociado: ', agent.trade_size)
      #agent.set_asset_and_order()
      msg = ''
      opened = agent.open_trade()
      
      if opened == False:
        print("asset não está disponivel no momento")
        print('indo para proxima negociação')
        continue
      else:
        print('negociação aberta com sucesso...iniciando streaming')
        agent.start_streaming()
        loss = agent.candle_analysis()

      # atualiza lucro ou perda geral do dia
      agent.set_profit_loss(loss)
      
      # entrando  no loop para loss=True/False
      # primeiro: se eu perdi, sempre vou abrir martingale! Sempre!
      if loss:
        opened = agent.open_martingale()
        mart_loss = agent.candle_analysis()
        agent.set_last_trade_won(False)

        # atualiza lucro ou perda geral do dia
        agent.set_profit_loss(mart_loss)

        if mart_loss:
          print('Número máximo de martingale atingido.')
          print(f'infelizmente você não ganhou no primeiro mart. no processo {process_number}')
          print( f'abrindo segundo martingale, no {process_number}')
          opened = agent.open_martingale()
          mart_loss = agent.candle_analysis()
          agent.set_last_trade_won(False)

          # atualiza lucro ou perda geral do dia
          agent.set_profit_loss(mart_loss)
          is_final_break = agent.is_final_break()
          if mart_loss:
            print('segundo martingale tbm perdeu')
          if is_final_break:
            break
        else:
          print(f'parabens você ganhou no primeiro martingale. no processo {process_number}')
          is_final_break = agent.is_final_break()
          if is_final_break:
            break
      
      # caso tenha ganhado, só abrirei soros se não tiver atingido a meta.    
      else:
        # parabens você ganhou esta negociação na proxima abriremos soros
        #if agent.last_trade_won:
        #  agent.set_last_trade_won(False)
        #else:
        #  agent.set_last_trade_won(True)
        # verifica se meta do dia foi atingida
        is_final_break = agent.is_final_break()
        if is_final_break:
          break
      
      # parando o streaming
      print('parando o streaming de preços...')
      agent.stop_streaming()
      # resetar valores antes do proximo loop
      # - resetar trade size
      # - resetar fator de multiplicação
      print( f'{process_number} resetando valores de negociação e martingale... ')
      agent.reset_defaults()
      print(f'{process_number} ajustando index da tabela de sinais...')
      agent.set_trade_index()
      print(f'{process_number} indo para a proxima análise no processo {process_number}... /n /n')

      ## problems: i dont have access to win pct of the normal binary
      ##           before open a trade. How to open soros in this situation?
  return msg

In [ ]:
agents_vector = []
for i in range(4):
  agent = TradeAgent(ds_vector[i], start_balance=balance, trade_type='digital',
                    initial_trade_size=2, stop_loss=-99999, duration=5,
                    max_profit=32)
  name = f'agent_{i}'
  agent.set_name(name)
  agents_vector.append(agent)

In [ ]:
import concurrent.futures

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = []
    for i in range(4):
      results.append(executor.submit(open_process, ds_vector[i], agents_vector[i]))

the inicial balance:  61.96
the inicial balance:  61.96
the inicial balance:  61.96
the inicial balance:  61.96
computing seconds left in process agent_0...
agent_0 sleeping for: 170.0 seconds. 
agent_0 sleeping for: 59.0 seconds. 
iniciando negociação normal
valor negociado:  2
- /n
previous balance:  3915.35
executing trade...
digital option...
option: GBPCAD-2-call-5
digital option requested
this is the trade id:  11517634962
ordem aberta. ID:  11517634962  - pct win:  0.85
negociação aberta com sucesso...iniciando streaming
================== /n
Agent_agent_0 Iniciando Stream de preços às:  time.struct_time(tm_year=2020, tm_mon=4, tm_mday=10, tm_hour=4, tm_min=40, tm_sec=5, tm_wday=4, tm_yday=101, tm_isdst=0)
Agent_agent_0 analisando vela...  /n
/n Agent_agent_0 verificando vitória ou derrota...
Agent_agent_0 você ganhou essa negociação.
current profit/loss:  1.7
parando o streaming de preços...
agent_0 resetando valores de negociação e martingale... 
agent_0 ajustando index da tab

In [ ]:
iqapi.change_balance('PRACTICE')
iqapi.get_balance()

3901.25

In [ ]:
id = iqapi.buy_digital_spot('EURUSD',1,'call',1)

In [ ]:
open_price = iqapi.get_async_order(id[1])

In [ ]:
open_price['open_underlying_price']

{}

In [ ]:
open_price['position-changed']['msg']['open_quote']

1.08161

In [ ]:
import concurrent.futures
import time
import multiprocessing

def do_something(seconds):
    print('sleeping for: ', seconds)
    time.sleep(seconds)
    return f'done sleeping..{seconds}'

start = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    seconds = [2, 10, 5]

    result = [executor.submit(do_something, sec) for sec in seconds]

    for f in concurrent.futures.as_completed(result):
        print(f.result())

finish = time.perf_counter()

print('finished in: ', start - finish)



sleeping for:  10
sleeping for:  2
sleeping for:  5
done sleeping..2
done sleeping..5
done sleeping..10
finished in:  -10.03250889200001


In [ ]:
ds_vector[0]

,time,duration,asset,order,hour,min,sec
0,12:52:00,1,CADJPY,PUT,12.0,52.0,0.0
1,12:52:00,1,EURGBP,CALL,12.0,52.0,0.0
2,12:52:00,1,USDCAD,CALL,12.0,52.0,0.0
3,08:55:00,1,EURCHF,CALL,8.0,55.0,0.0
4,08:57:00,1,EURCHF,CALL,8.0,57.0,0.0
5,10:30:00,1,EURCAD,CALL,10.0,30.0,0.0
6,10:30:00,1,USDCAD,CALL,10.0,30.0,0.0
7,10:30:00,1,USDCHF,PUT,10.0,30.0,0.0
8,10:30:00,1,USDJPY,PUT,10.0,30.0,0.0
9,10:32:00,1,EURCAD,CALL,10.0,32.0,0.0


### Debug

In [ ]:
print('time before: ', time.localtime())
info = iqapi.get_digital_current_profit('EURUSD-OTC',60)
print('time after: ', time.localtime())

time before:  time.struct_time(tm_year=2020, tm_mon=3, tm_mday=14, tm_hour=17, tm_min=36, tm_sec=50, tm_wday=5, tm_yday=74, tm_isdst=0)
time after:  time.struct_time(tm_year=2020, tm_mon=3, tm_mday=14, tm_hour=17, tm_min=36, tm_sec=50, tm_wday=5, tm_yday=74, tm_isdst=0)


In [ ]:
cc=iqapi.get_realtime_candles(agent.curr_asset, agent.size_streaming)

In [ ]:
cc[1584280232]['to']

1584280233

In [ ]:
int(datetime.fromtimestamp(cc[1584280232]['from']).strftime('%M'))

50

In [ ]:
time.localtime()

time.struct_time(tm_year=2020, tm_mon=3, tm_mday=15, tm_hour=11, tm_min=2, tm_sec=14, tm_wday=6, tm_yday=75, tm_isdst=0)

In [ ]:
strftime

In [ ]:
goal = 'AUDJPY'
size = 1

loop = True
loop2 = True
while loop:
  #if time.localtime().tm_sec % 10 == 0:
  print('================== /n')
  print(time.localtime().tm_sec)

  iqapi.start_candles_stream(goal,size, 1)

  print("print candles")
  while loop2:
    cc=iqapi.get_realtime_candles(goal,size)
    for k in cc:
      if int(datetime.fromtimestamp(cc[k]['to']).strftime("%S")) % 10 == 0:
        print(goal,"size",k,cc[k])
        print('current time executed: ', time.localtime().tm_sec)
        print('the current seconds on the server: ', int(datetime.fromtimestamp(cc[k]['to']).strftime("%S")))
    time.sleep(1)
  print("stop candle")
  iqapi.stop_candles_stream(goal,size)
  loop = False

In [ ]:
_, id = iqapi.buy(1, 'EURUSD-OTC', 'call', 1)

In [ ]:
id = iqapi.buy_digital_spot('EURCHF-OTC', 1, 'call', 5)

In [ ]:
id

'error'

In [ ]:
order_dict = iqapi.get_async_order(id)

In [ ]:
order_dict

In [ ]:
agent.curr_trade_id

0

In [ ]:
# after waking up Lets trade!!!
    if agent.last_trade_won:
      print('iniciando negociação soros')
      print('valor negociado: ', agent.trade_size)
      #agent.set_asset_and_order()
      opened = agent.open_soros()
      agent.start_streaming()
      loss = agent.candle_analysis()
    else:

elif do_soros:
        print('-------')
        print('meta do dia ou stop loss não alcançados ainda.')
        print('continuaremos a negociar.')
        print('checking for soros...')
        agent.open_soros()
        soros_loss = agent.candle_analysis()

        # atualiza lucro ou perda geral do dia
        agent.set_profit_loss(soros_loss)
        if soros_loss:
          print( 'infelizmetne vc perdeu no soros')
          # se eu perco no soros, tenho que abrir martingale normal
          agent.reset_trade_size()
          opened = agent.open_martingale()
          if opened == False:
            print('parando while loop pois instrumento não existe')
            agent.reset_defaults()
            continue
          mart_loss = agent.candle_analysis()

          # atualiza lucro ou perda geral do dia
          agent.set_profit_loss(mart_loss)

          if mart_loss:
            print('Número máximo de martingale atingido.')
            print('infelizmente você não ganhou dessa vez.')
            is_final_break = agent.is_final_break()
            if is_final_break:
              break
          else:
            print('parabens você ganhou no martingale.')
            is_final_break = agent.is_final_break()
            if is_final_break:
              break
        else:
          print('parabens você ganhou no soros')
        # verifica se meta do dia foi atingida
        is_final_break = agent.is_final_break()
        if is_final_break:
          agent.reset_defaults()
          break
      elif do_soros == False:


### Version update history
Version 0.1
  * correção de bug verifica se a trade foi aberta
  * correção na duração da trade para diferentes timeframes